# Filtering in spark, cache(), and persist()
finding similar movies using spark and the MovieLens data set

Introducing caching RDD's

### How to do it ( _this is just one way to do it!_ )
- Find every pair of movies that were watch by the same person
- Measure the similarity of their ratings across all users who watched both
- Sort by movie, then by similarity strength

## Making it a spark problem!
- map input ratings to (user, (movieID, rating))
- find every movie pair rated by the same user
    - this can be done with a "self-join" operation
    - At this point we have (userID, ((movieID1, rating1), (movieID2,rating2)))
- Filter out duplicate pairs
- Make the movie pairs the key
    - map to ((movieID1, movieID2), (rating1, rating2))
- groupByKey() to get every rating pair found for each movie pair
- Compute similarity between ratings for each movie in the pair
- Sort, save, and display the results

## Using all our pc's cores
`sc = SparkContext("local[*]", "MovieSimilarities")`

## Caching RDD's
- In this example, we'll query the final RDD of movie similarities a couple of times
- Any time you will perform more than one action on an RDD, you must cache it!
- Use .cache() or .persist() to do this.
    - Whats the difference?
    - Persist() optionally lets you cache it to disk instead of just memory, just in case a node fails or something

In [1]:
import sys
from pyspark import SparkContext
from math import sqrt

#import os 
#os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
#os.environ["PYSPARK_PYTHON"] = "/usr/local/bin/python3"
#print ('PYLIB: ', os.environ.get('PYLIB'))
#print ('PYSPARK_PYTHON', os.environ.get('PYSPARK_PYTHON'))

sc = SparkContext("local[*]", "G")

In [2]:
def loadMovieNames():
    movieNames = {}
    with open("../data/u.item", encoding='ascii', errors='ignore') as f:
        for line in f:
            fields = line.split('|')
            movieNames[int(fields[0])] = fields[1]
    return movieNames

#Python 3 doesn't let you pass around unpacked tuples,
#so we explicitly extract the ratings now.
def makePairs( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return ((movie1, movie2), (rating1, rating2))

def filterDuplicates( userRatings ):
    ratings = userRatings[1]
    (movie1, rating1) = ratings[0]
    (movie2, rating2) = ratings[1]
    return movie1 < movie2

def computeCosineSimilarity(ratingPairs):
    numPairs = 0
    sum_xx = sum_yy = sum_xy = 0
    for ratingX, ratingY in ratingPairs:
        sum_xx += ratingX * ratingX
        sum_yy += ratingY * ratingY
        sum_xy += ratingX * ratingY
        numPairs += 1

    numerator = sum_xy
    denominator = sqrt(sum_xx) * sqrt(sum_yy)

    score = 0
    if (denominator):
        score = (numerator / (float(denominator)))

    return (score, numPairs)

In [3]:

print("\nLoading movie names...")
nameDict = loadMovieNames()

data = sc.textFile("../data/u.data")

# Map ratings to key / value pairs: user ID => movie ID, rating
ratings = data.map(lambda l: l.split()).map(lambda l: (int(l[0]), (int(l[1]), float(l[2]))))

# Emit every movie rated together by the same user.
# Self-join to find every combination.
joinedRatings = ratings.join(ratings)

# At this point our RDD consists of userID => ((movieID, rating), (movieID, rating))

# Filter out duplicate pairs
uniqueJoinedRatings = joinedRatings.filter(filterDuplicates)

# Now key by (movie1, movie2) pairs.
moviePairs = uniqueJoinedRatings.map(makePairs)

# We now have (movie1, movie2) => (rating1, rating2)
# Now collect all ratings for each movie pair and compute similarity
moviePairRatings = moviePairs.groupByKey()

# We now have (movie1, movie2) = > (rating1, rating2), (rating1, rating2) ...
# Can now compute similarities.
moviePairSimilarities = moviePairRatings.mapValues(computeCosineSimilarity).cache()

# Save the results if desired
# moviePairSimilarities.sortByKey()
# moviePairSimilarities.saveAsTextFile("movie-sims")

# Extract similarities for the movie we care about that are "good".
if (len(sys.argv) > 1):

    scoreThreshold = 0.97
    coOccurenceThreshold = 50

    movieID = 50 # int(sys.argv[1])

    # Filter for movies with this sim that are "good" as defined by
    # our quality thresholds above
    filteredResults = moviePairSimilarities.filter(lambda pairSim: \
        (pairSim[0][0] == movieID or pairSim[0][1] == movieID) \
        and pairSim[1][0] > scoreThreshold and pairSim[1][1] > coOccurenceThreshold)

    # Sort by quality score.
    results = filteredResults.map(lambda pairSim: (pairSim[1], pairSim[0])).sortByKey(ascending = False).take(10)

    print("Top 10 similar movies for " + nameDict[movieID])
    for result in results:
        (sim, pair) = result
        # Display the similarity result that isn't the movie we're looking at
        similarMovieID = pair[0]
        if (similarMovieID == movieID):
            similarMovieID = pair[1]
        print(nameDict[similarMovieID] + "\tscore: " + str(sim[0]) + "\tstrength: " + str(sim[1]))



Loading movie names...
Top 10 similar movies for Star Wars (1977)
Empire Strikes Back, The (1980)	score: 0.9895522078385338	strength: 345
Return of the Jedi (1983)	score: 0.9857230861253026	strength: 480
Raiders of the Lost Ark (1981)	score: 0.981760098872619	strength: 380
20,000 Leagues Under the Sea (1954)	score: 0.9789385605497993	strength: 68
12 Angry Men (1957)	score: 0.9776576120448436	strength: 109
Close Shave, A (1995)	score: 0.9775948291054827	strength: 92
African Queen, The (1951)	score: 0.9764692222674887	strength: 138
Sting, The (1973)	score: 0.9751512937740359	strength: 204
Wrong Trousers, The (1993)	score: 0.9748681355460885	strength: 103
Wallace & Gromit: The Best of Aardman Animation (1996)	score: 0.9741816128302572	strength: 58
